In [ ]:
### Dataset Analysis
from thermo_validity import validity
import numpy as np
from split_functions import equipments_to_strings
cutoff = 143.957
# cutoff = np.inf
p_dataset = np.load("GPT_NA_psitest/m2_data_8.npy", allow_pickle=True)
# generated = np.load("GPT_NA_psitest/generated_M2_2.npy",allow_pickle=True)
# results = np.load("GPT_NA_psitest/results_candidates_2.npy")
data = np.load("MCTS_inference/MCTS_generated_designs_psi_2_500_0.03_0.5.npy", allow_pickle=True)
values = data[:,0]
designs = data[:,1]
results = [300-175*x for x in values]
results = np.asanyarray(results)
generated = equipments_to_strings(designs)
print(len(p_dataset))
valid = validity(generated)
print(len(valid))
unique = np.unique(valid)
print(len(unique))
# datalist = unique
# n_dataset = np.unique(np.concatenate((p_dataset, unique), axis=0))
# datalist = n_dataset[np.where(np.isin(n_dataset, p_dataset, invert=True))[0]]
datalist = generated
print(len(datalist), len(results))
nonzero_results = results[np.where(results > 0)]
good_layouts = []
good_results = []
print("Optimization Results:", len(nonzero_results), len(results))
for i in range(len(results)):
    if results[i] < cutoff and results[i] > 0:
        good_layouts.append(datalist[i])
        good_results.append(results[i])
print("Good layouts", len(good_layouts))
good_layouts = np.asanyarray(good_layouts)
good_results = np.asanyarray(good_results)
indices = np.argsort(good_results)
sorted_results = good_results[indices]
sorted_layouts = good_layouts[indices]
print(sorted_results[:10], sorted_layouts[:10])
print("Mean of the good results:", np.mean(good_results))
print("Std of the good results:", np.std(good_results))
print("Mean non-zero results:", np.mean(nonzero_results))

In [ ]:
from mcts_util import *
model = LSTM()
args = {
    "C": 1.5,
    "num_searches": 200,
    "num_iterations": 25,
    "num_selfPlay_iterations": 400,
    "num_epochs": 5,
    "batch_size": 100,
    "temperature": 1,
    "dirichlet_epsilon": 0.03,
    "dirichlet_alpha": 0.5,
    "save_path": "./RL/policy_value_model_trials",
}
# inference
fw = Flowsheet()
model.load_state_dict(torch.load("RL/policy_value_model_trials/model_24_Flowsheet.pt"))
mcts = MCTS(fw, args, model)
best_value = 0
generated_designs = []
for i in range(10):
    state = fw.get_initial_state()
    while True:
        mcts_probs = mcts.search(state)
        action = np.argmax(mcts_probs)
        state = fw.get_next_state(state, action)
        value, is_terminal = fw.get_value_and_terminated(state, action)

        if is_terminal:
            if value > best_value:
                best_value = value
            if i % 50 == 0:
                print(i, "Best Value Found:", best_value)
            generated_designs.append((value, fw.get_encoded_state(state)))
            break

valid_designs = [layout for value, layout in generated_designs if value > 0]
for v, l in valid_designs:
    print(v, l)
print(len(valid_designs))

In [ ]:
from ZW_utils import std_classes
layouts = np.load("M2_data_300_8_augmented_layouts.npy", allow_pickle=True)
results = np.load("M2_data_300_8_augmented_results.npy", allow_pickle=True)
layouts = equipments_to_strings(layouts, std_classes)
# layouts = ["GTACH","GTACH","GTACH","GTACH","GTACH","GTACHE"]
# results = [0.3,0.5,0.2,0.1,0.7,1]
#sort results use indices to sort layouts
indices = np.argsort(results)
sorted_results = np.array(results)[indices]
sorted_layouts = np.array(layouts)[indices]
unique,indices = np.unique(sorted_layouts, return_index=True)
unique_results = sorted_results[indices]
unique_layouts = sorted_layouts[indices]
sorted_results[:100]

In [3]:
import torch
import torch.nn as nn
import numpy as np
from split_functions import equipments_to_strings, classes
from thermo_validity import validity
std_classes = classes
class LSTMemb(nn.Module):
    def __init__(self, hidden_size=32, num_layers=2, out_features=12, emb_size=16):
        super().__init__()
        self.embedding = nn.Embedding(13, embedding_dim=emb_size, padding_idx=12)
        self.lstm = nn.LSTM(
            input_size=emb_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.policyhead = nn.Linear(in_features=hidden_size, out_features=out_features)
        self.valuehead = nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, x, lengths):
        x = self.embedding(x.squeeze(-1).long())
        # x_packed = nn.utils.rnn.pack_padded_sequence(
        #     x, lengths, batch_first=True, enforce_sorted=False
        # )
        output, (hidden, _) = self.lstm(x)
        policy = self.policyhead(hidden[-1])
        value = self.valuehead(hidden[-1])
        return policy, value
    
model = LSTMemb(hidden_size=256          ,emb_size=32)
model.load_state_dict(torch.load("RL/policy_value_model_trials/model_21_Flowsheet.pt"))
model.eval()
generated = []
x_0 = np.ones(22)*12
x_0[0] = 0
x_0 = torch.tensor(x_0).reshape(1, 22, 1)
for N in range(1000):
    x = x_0.clone()
    lengths = torch.tensor([1])
    while True:
        with torch.no_grad():
            policy, value = model(x, lengths)
            action = torch.multinomial(torch.softmax(policy, dim=-1), num_samples=1)
            x[0][lengths.item()] = action
            lengths += 1
            if action.item() == 11:
                break
            if lengths.item() == 22:
                break
    generated.append(x.squeeze())
generated = torch.stack(generated)
generated = generated.numpy().astype(int)
generated =[list(x[x != 12]) for x in generated]
generated = equipments_to_strings(generated, std_classes)
valid = validity(generated)
print("Valid",len(valid))
print("Unique Valid", len(np.unique(valid)))

Valid 576
Unique Valid 570


In [13]:
#MAX MIN REWARD CHECKS
import numpy as np
from split_functions import layout_to_string_single_1d
from thermo_validity import validity
def evaluation(layout):
    reward = 0
    layout = layout.astype(int)
    def reward_scaling(reward):
        current_max = 5.7
        current_min = -1
        desired_max = 1
        desired_min = -1
        return (
            (reward - current_min) / (current_max - current_min)
        ) * (desired_max - desired_min) + desired_min

    # no equipment repetition back to back
    for i in range(1, len(layout) - 2):
        if layout[i] != layout[i + 1]:
            reward += 0.1
    if layout[1] != layout[-2]:
        reward += 0.1
    # has an ending token
    if layout[-1] == 11:
        reward += 0.1
    # has the basic structure [1, 2, 3, 4]
    if 1 in layout and 2 in layout and 3 in layout and 4 in layout:
        reward += 0.4
    # if it has a hx, it has 2 of them
    if 5 in layout:
        if np.count_nonzero(layout == 5) == 2:
            reward += 0.6
    # if it has a splitter, it has 1 of them
    if 9 in layout:
        if np.count_nonzero(layout == 9) == 1:
            reward += 0.2
        if np.count_nonzero(layout == 7) == 2:
            reward += 0.4
    # validity check
    stringlist = [
        layout_to_string_single_1d(layout),
    ]
    valid_string = validity(stringlist)
    reward = reward + 1 if len(valid_string) > 0 else reward - 1
    print(reward)
    if len(valid_string) == 0:
        return reward_scaling(reward)
    else:
        value = 1
        return reward_scaling(reward + value)
layout = np.array([0,1,2,3,5,4,5,7,1,9,1,2,3,4,7,1,2,3,4,2,3,11])
print(evaluation(layout))
layout = np.array([0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
print(evaluation(layout))

4.700000000000001
1.0000000000000004
-1
-1.0


In [ ]:
from ZW_Opt import *
from split_functions import (
    bound_creation,
    layout_to_string_single_1d,
    equipments_to_strings,
)
from thermo_validity import *
classes = std_classes
layouts = np.load("M2_data_300_8_augmented_layouts.npy", allow_pickle=True)
results = np.load("M2_data_300_8_augmented_results.npy", allow_pickle=True)
print(len(layouts), len(results))
layouts = equipments_to_strings(layouts, classes)
results = 1 - (results - 125) / 175
indices = np.argsort(results)
sorted_results = np.array(results)[indices]
sorted_layouts = np.array(layouts)[indices]
unique, indices = np.unique(sorted_layouts, return_index=True)
unique_results = sorted_results[indices]
unique_layouts = sorted_layouts[indices]
print(len(unique_layouts), len(unique_results))
layouts = unique_layouts.tolist()
results = unique_results
new_layouts = []
new_results = []

def evaluation(layout):
    reward = 0
    layout = layout.astype(int)

    # no equipment repetition back to back
    for i in range(1, len(layout) - 2):
        if layout[i] != layout[i + 1]:
            reward += 0.1
    if layout[1] != layout[-2]:
        reward += 0.1
    # has an ending token
    if layout[-1] == 11:
        reward += 0.1
    # has the basic structure [1, 2, 3, 4]
    if 1 in layout and 2 in layout and 3 in layout and 4 in layout:
        reward += 0.4
    # if it has a hx, it has 2 of them
    if 5 in layout:
        if np.count_nonzero(layout == 5) == 2:
            reward += 0.2
    # if it has a splitter, it has 1 of them
    if 9 in layout:
        if np.count_nonzero(layout == 9) == 1:
            reward += 0.1
        if np.count_nonzero(layout == 7) == 2:
            reward += 0.2
    # validity check
    stringlist = [
        layout_to_string_single_1d(layout),
    ]
    valid_string = validity(stringlist)
    reward = reward + 1 if len(valid_string) > 0 else reward - 1
    
    
    if len(valid_string) == 0:
        #scale validitiy rewards -1,4 to -1,0
        reward = -1 + (reward - (-1)) * (0 - (-1)) / (4 - (-1))
        return reward    

    if valid_string[0] in new_layouts:
        value = new_results[new_layouts.index(valid_string[0])]
        return value
    if valid_string[0] in layouts:
        value = results[layouts.index(valid_string[0])]
        return value

    ohe = np.zeros((len(layout), len(classes)))
    for i, l in enumerate(layout):
        ohe[i, l] = 1
    equipment, bounds, x, splitter = bound_creation(ohe)
    swarmsize_factor = 7
    nv = len(bounds)
    particle_size = swarmsize_factor * nv
    if 5 in equipment:
        particle_size += -1 * swarmsize_factor
    if 9 in equipment:
        particle_size += -2 * swarmsize_factor
    iterations = 30
    try:
        a = PSO(objective_function, bounds, particle_size, iterations, nv, equipment)
        if a.result < 300:
            # standardization between 125 and 300 to 1 and 0.5
            value = 1 - (a.result - 125)*(1-0.5) / 175
            new_layouts.append(valid_string[0])
            new_results.append(value)
        elif a.result < 1e6:
            #standardization between 300 and 1e6 to 0.5 and 0.25
            value = 0.5 - (a.result - 300)*(0.5-0.25) / (1e6-300)
            new_layouts.append(valid_string[0])
            new_results.append(value)
        else:
            value = 0+(reward - (-1)) * (0.25 - 0) / (4 - (-1))
    except:
        value = 0
    return value
layout =[0, 3, 1, 3, 1, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1, 7, 2, 9, 3, 7, 11]

print(evaluation(np.array(layout)))


In [ ]:
1 - (135 - 125)*(1-0.5) / 175

In [ ]:
import torch
import torch.nn as nn

class LSTM_packed(nn.Module):
    def __init__(self, embd_size,hidden_size):
        super(LSTM_packed, self).__init__()
        self.embedding = nn.Embedding(13, embd_size, padding_idx=12)
        self.lstm = nn.LSTM(embd_size, hidden_size, num_layers=2, batch_first=True, dropout=0.1)
        self.valuehead = nn.Linear(hidden_size, 1)
        self.policyhead = nn.Linear(hidden_size, 12)
    def forward(self, x, lengths):
        
        x = self.embedding(x.long())
        x = nn.utils.rnn.pack_padded_sequence(
            x, lengths, batch_first=True, enforce_sorted=False
        )
        output, (hidden, _) = self.lstm(x)
        value = self.valuehead(hidden[-1])
        policy = self.policyhead(hidden[-1])
        return policy, value

model = LSTM_packed(64,256)
model.load_state_dict(torch.load("RL/policy_value_model_parallel/model_2_Flowsheet.pt"))
model.eval()
x = torch.tensor([0]).reshape(1,1)
lengths = torch.tensor(x.shape[1]).reshape(1)
while x.shape[1] < 23:
    policy, value = model(x,lengths)
    # next_token = torch.multinomial(nn.Softmax(dim=-1)(policy),1).reshape(-1)
    next_token = torch.argmax(nn.Softmax(dim=-1)(policy),dim=-1)
    x = torch.cat((x,next_token.reshape(1,1)),dim=1)
    if next_token.item() ==11:
        break
    lengths = torch.tensor(x.shape[1]).reshape(1)
print(x,value)